# recuperation des donné avec des fonctions asynchrones pour plus de performances

In [1]:
import asyncio
import requests
import pandas as pd
import aiohttp
import time


In [2]:

async def fetch_data(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.json()


In [6]:
datacount_base_url = f"https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-data/records"
counter_location_base_url = f"https://data.metropole-rouen-normandie.fr/api/explore/v2.1/catalog/datasets/eco-counter-sites/records"


In [6]:

async def get_data_as_dataframe(
        limit : int = 50, 
        offset : int = 0, 
        select : str = "", 
        where  : str  = "",
        group_by : str = "",
        order_by : str = "",
        refine : dict = {},
        exclude : dict = {}
        )  -> pd.DataFrame:
    

    # query = f"?limit={limit}" \
    #         f"&offset={offset}" \
    query = ""
    if select != "":
        query += f"{'?' if len(query) == 0 else '&'}select={select}" 
    
    if where != "":
        query += f"{'?' if len(query) == 0 else '&'}where={where}" 

    if group_by != "":
        query += f"{'?' if len(query) == 0 else '&'}group_by={group_by}" 

    if order_by != "":
        query += f"{'?' if len(query) == 0 else '&'}order_by={order_by}" 

    if refine != {}:
        for key in refine:
            query += f"{'?' if len(query) == 0 else '&'}refine={key}%3A{refine[key]}"

    if exclude != {}:
        for key in exclude:
            query += f"{'?' if len(query) == 0 else '&'}exclude={key}%3A{exclude[key]}"
        
    if 0 < limit <= 100:
        query += f"{'?' if len(query) == 0 else '&'}limit={limit}" \
            f"&offset={offset}" 
        response = requests.get(datacount_base_url + query).json()
        total_count = response["total_count"] 
        data = response["results"]

    else:
        if limit == 0:
            response = requests.get(datacount_base_url + query+ f"{'?' if len(query) == 0 else '&'}limit=1").json()
            limit = int(response["total_count"])
        # le nombre de requete de limité à 100 entrés possible
        nbre_request = limit // 100
        # le nombre d'entré de la derniere requete
        last_limit = limit  % 100
        
        data = []
        start = time.perf_counter()
        for idx_req in range(nbre_request):
            limit_query = f"{'?' if len(query) == 0 else '&'}limit={100}" \
                    f"&offset={offset + idx_req*100}"
            response = await fetch_data(datacount_base_url + query + limit_query )# requests.get().json() 
            if "message" in response.keys():
                print(response)
                break
            data += response["results"]
        
        #envoie de la dernier requete
        limit_query = f"{'?' if len(query) == 0 else '&'}limit={last_limit}" \
                f"&offset={offset + (nbre_request)*100}"
        response = await fetch_data(datacount_base_url + query + limit_query ) #requests.get(datacount_base_url + query + limit_query).json() 
        data += response["results"]
        print("Time", time.perf_counter - start)
        total_count = response["total_count"] 
    return f"{time.perf_counter - start}", pd.DataFrame(data)



async def main():
    time, df = await get_data_as_dataframe(9000)
    print(time)

loop = asyncio.get_event_loop()
loop.create_task( main())

<Task pending name='Task-9' coro=<main() running at /var/folders/v6/gjxkdclj08sbj2vczb74wtxr0000gn/T/ipykernel_959/3572965771.py:74>>

In [5]:
loop = asyncio.get_event_loop()
loop.create_task( main())

<Task pending name='Task-8' coro=<main() running at /var/folders/v6/gjxkdclj08sbj2vczb74wtxr0000gn/T/ipykernel_959/3572965771.py:74>>